In [1]:
from geopy.geocoders import GoogleV3
import numpy as np
import geocoder
import json
import pandas as pd
from geojson import Feature, FeatureCollection, Point
import tabula
import re
import os

In [2]:
# Gotta figure out how to get rid of the java error.
# SO: https://stackoverflow.com/questions/54817211/java-command-is-not-found-from-this-python-process-please-ensure-java-is-inst

dsf = tabula.io.read_pdf('condo.pdf', pages='all',lattice=True)

In [3]:
df = dsf[0]
df.columns = df.iloc[0]
df = df.drop(df.index[0])
### This next line gets ride of the Nan ###
df = df.iloc[: , 1:]
df = df.replace(np.nan, 'Not Available', regex=True)
df = df.rename(columns={'Building / Address / City': 'building_address_city','Days on\rMarket':'days_on_market',\
                        'Price /\rSq. Ft.':'price_per_sqft'})
df['geo_address'] = df['building_address_city']
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df2 = df['geo_address'].str.extract(r'(?P<building_name>\D*)(?P<address_only>\d.*)', expand=True)

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [4]:
result = pd.merge(df,df2,left_index=True, right_index=True)
df = result

In [5]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [6]:
df['loc'] = df['address_only'].apply(geolocator.geocode, timeout=10)

In [7]:
df['loc']

1     (2675 S Bayshore Dr 901 s, Miami, FL 33133, US...
2     (1000 Biscayne Blvd #3601, Miami, FL 33132, US...
3     (3801 Collins Ave #606, Miami Beach, FL 33140,...
4     (450 Alton Rd APT 2902, Miami Beach, FL 33139,...
5     (5959 Collins Ave APT 1701, Miami Beach, FL 33...
6     (801 South Pointe Dr APT 202, Miami Beach, FL ...
7     (17121 Collins Ave APT 2607, Sunny Isles Beach...
8     (21150 NE 38th Ave APT 2304, Aventura, FL 3318...
9     (901 Brickell Key Blvd APT 2707, Miami, FL 331...
10    (90 Alton Rd #4, Miami Beach, FL 33139, USA, (...
Name: loc, dtype: object

In [8]:
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)

Columns must be same length as key explained
- https://cumsum.wordpress.com/2021/06/01/pandas-valueerror-columns-must-be-same-length-as-key/

Creating `df2` based on `df` row indicies.

https://datascience.stackexchange.com/questions/77033/creating-a-new-dataframe-with-specific-row-numbers-from-another

What's tripping this error is when the lat/lon/alititude generator is fed a Nan value to parse - it's effectively trying to split a `null` value into three separate parts - which is impossible.

The code below effectively caputres the anatomy of this error. It creates a new `df` based on a list of indicies from an existing one. Row `56` has no `point` since the `geocoder` couldn't find coordinates for it. The other three do. Splitting the `point` column fails only when row `56` - with `null` values - is attempted.
```
df2 = df.iloc[[56,57,58,59],:]
df2[['lat','lon','altitude']] = pd.DataFrame(df2['point'].to_list(),index=df2.index)
```

In [9]:
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

df

,building_address_city,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only,loc,point,lat,lon,altitude
1,Grove at Grand Bay\r2675 S Bayshore Dr 901-S\r...,1/9/2023,137,"$7,600,000.00",4647,"$1,635.46",Sandra Masis,Cervera Real Estate Inc.,Isabel Castro,BHHS EWM Realty,Grove at Grand Bay 2675 S Bayshore Dr 901-S Co...,Grove at Grand Bay,2675 S Bayshore Dr 901-S Coconut Grove,"(2675 S Bayshore Dr 901 s, Miami, FL 33133, US...","(25.729886, -80.2368583, 0.0)",25.729886,-80.236858,0.0
2,One Thousand Museum\r1000 Biscayne Blvd 3601\r...,1/13/2023,237,"$7,600,000.00",5576,"$1,558.65",Patricia Zimmermann,Great Estates Int'l Realty,Dina Goldentayer,Douglas Elliman,One Thousand Museum 1000 Biscayne Blvd 3601 Miami,One Thousand Museum,1000 Biscayne Blvd 3601 Miami,"(1000 Biscayne Blvd #3601, Miami, FL 33132, US...","(25.784171, -80.1900783, 0.0)",25.784171,-80.190078,0.0
3,Mosaic\r3801 Collins Ave 606\rMiami Beach,1/9/2023,94,"$3,300,000.00",1820,"$1,813.19",Steven Koleno,Beycome of Florida LLC,Non Member Listing Agent,Non Member Listing Office,Mosaic 3801 Collins Ave 606 Miami Beach,Mosaic,3801 Collins Ave 606 Miami Beach,"(3801 Collins Ave #606, Miami Beach, FL 33140,...","(25.8113373, -80.12262849999999, 0.0)",25.811337,-80.122628,0.0
4,Icon South Beach\r450 Alton Rd 2902\rMiami Beach,1/12/2023,3,"$3,150,000.00",1933,"$1,629.59",Giorgio Vecchi,Douglas Elliman,Fabrizio Pellicciotti,"Miami Blue Real Estate, LLC",Icon South Beach 450 Alton Rd 2902 Miami Beach,Icon South Beach,450 Alton Rd 2902 Miami Beach,"(450 Alton Rd APT 2902, Miami Beach, FL 33139,...","(25.7719135, -80.1380353, 0.0)",25.771914,-80.138035,0.0
5,The Bath Club\r5959 Collins Ave 1701\rMiami Beach,1/10/2023,29,"$2,440,000.00",2059,"$1,185.04",Joanne Jones,Compass Florida LLC,Krisztian Nagy,Keller Williams Miami Beach Realty,The Bath Club 5959 Collins Ave 1701 Miami Beach,The Bath Club,5959 Collins Ave 1701 Miami Beach,"(5959 Collins Ave APT 1701, Miami Beach, FL 33...","(25.8424476, -80.1203048, 0.0)",25.842448,-80.120305,0.0
6,Marea South Beach\r801 S Pointe Dr 202\rMiami ...,1/13/2023,162,"$2,100,000.00",1732,"$1,212.47",Geane Brito,Compass Florida LLC,Natalia Figueroa,Compass Florida LLC,Marea South Beach 801 S Pointe Dr 202 Miami Beach,Marea South Beach,801 S Pointe Dr 202 Miami Beach,"(801 South Pointe Dr APT 202, Miami Beach, FL ...","(25.7686007, -80.1361615, 0.0)",25.768601,-80.136162,0.0
7,Jade Ocean\r17121 Collins Ave 2607\rSunny Isle...,1/10/2023,78,"$1,620,000.00",1161,"$1,395.35",Romina Kotliar,Compra Venta Miami,Maria Kunigonis,Compass Florida LLC,Jade Ocean 17121 Collins Ave 2607 Sunny Isles ...,Jade Ocean,17121 Collins Ave 2607 Sunny Isles Beach,"(17121 Collins Ave APT 2607, Sunny Isles Beach...","(25.9343529, -80.1210992, 0.0)",25.934353,-80.121099,0.0
8,Atlantic II\r21150 NE 38 Ave 2304\rAventura,1/10/2023,71,"$1,550,000.00",2650,$584.91,Priscila Camargo,Elite Sales Group,Gustavo Blachman,Terrabella Realty,Atlantic II 21150 NE 38 Ave 2304 Aventura,Atlantic II,21150 NE 38 Ave 2304 Aventura,"(21150 NE 38th Ave APT 2304, Aventura, FL 3318...","(25.9722404, -80.1252914, 0.0)",25.972240,-80.125291,0.0
9,Carbonell\r901 Brickell Key Blvd 2707\rMiami,1/10/2023,93,"$1,350,000.00",1784,$756.73,Eric Lastra,Image Setters Realty,Tatiana Costa-Martinez,Fortune Christie's Int'l Real Estate,Carbonell 901 Brickell Key Blvd 2707 Miami,Carbonell,901 Brickell Key Blvd 2707 Miami,"(901 Brickell Key Blvd APT 2707, Miami, FL 331...","(25.7690569, -80.186713, 0.0)",25.769057,-80.186713,0.0
10,0Yacht Club at Portofino\r90 Alton Rd FL-4\rMi...,1/13/2023,58,"$1,225,000.00",1070,"$1,144.86",David Veit PA,Brown Harris Stevens,Todd Nordstrom,Compass Florida LLC,0Yacht Club at Portofino 90 Alton Rd FL-4 Miam...,,0Yacht Club at Portofino 90 Alton Rd FL-4 Miam...,"(90 Alton Rd #4, Miami Beach, FL 33139, USA, (...","(25.769951, -80.1382778, 0.0)",25.769951,-80.138278,0.0


Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

In [10]:
df.at[10,'building_name']=('Yacht Club at Portofino')
df.at[10,'address_only']=('90 Alton Rd')

In [11]:
df.columns

Index(['building_address_city', 'Close Date', 'days_on_market', 'Sale Price',
       'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker', 'Buyer Agent',
       'Buyer Broker', 'geo_address', 'building_name', 'address_only', 'loc',
       'point', 'lat', 'lon', 'altitude'],
      dtype='object')

In [12]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=10)
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('January 8th - January 14th')

for i,r in df.iterrows():
    location = (r["lat"], r["lon"])
    info = (r["Sale Price"], r["days_on_market"], r["Agent"], r['Buyer Agent'], r['price_per_sqft'], r['address_only'], r['building_name'])
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'Building Name: {bold_start}{info[-1]}{bold_end}{new_line} \
    Sale Price: {bold_start}{info[0]}{bold_end}{new_line}Days on Market: \
    {bold_start}{info[1]}{bold_end}{new_line}Listing Agent: {bold_start}{info[2]} \
    {bold_end}{new_line}Buying Agent: {bold_start}{info[3]}{bold_end}{new_line}Price sq ft: \
    {bold_start}{info[4]}{bold_end}{new_line}Address: {bold_start}{info[5]}{bold_end}'
    folium.Marker(location=location,
                      tooltip=text)\
    .add_to(m)

m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))

m

In [13]:
m.save('index.html')

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [14]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [15]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/condo_sales_week_ending_011622
